<a href="https://colab.research.google.com/github/DenisFerreira/Pesquisa_Operacional/blob/main/Problema_do_fluxo_Maximo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Otimização em Rede

# Problema do fluxo Máximo

*   Imagine uma rede de distribuição de petróleo entre poços e refinarias
*   Entre os poços e as refinarias existem estações intermediárias para ajudar a impulsionar o fluxo
*   Os poços, refinarias e estações intermediárias são ligados por tubulações que possuem capacidade máxima de fluxos.

O Objetivo deste problema é maximizar o fluxo de fluido dos poços até as refinarias

Restrito a:
*   No nó de origem só existe saída de fluxo
*   No nó de destino só existe entrada de fluxo
*   Nos demais nós o fluxo de entrada deve ser igual ao de saída
*   A capacidade de escoamento das tubulações deve ser respeitada


In [1]:
!!pip install pulp
from pulp import *

In [11]:
# Dados do problema
n_nos = [0, 1, 2, 3, 4]
no_origem =  n_nos[0]
no_destino = n_nos[4]

grafo = [
         [0,1,1,1,0],
         [0,0,1,0,1],
         [0,0,0,1,1],
         [0,0,1,0,1],
         [0,0,0,0,0]
         ]

fluxo_maximo = [
          [0,200,300,100,0],
          [0,0,400,0,300],
          [0,0,0,100,200],
          [0,0,50,0,200],
          [0,0,0,0,0]
]


# Variáveis de decisão
var = {}
for i in n_nos:
  for j in n_nos:
    if grafo[i][j] == 1 :
      # As variáveis representam o fluxo que será utilizado em cada aresta
      var[(i,j)] = LpVariable(name=f'x{i}{j}', lowBound=0, upBound=fluxo_maximo[i][j], cat=LpContinuous)

# Criação do Modelo
model = LpProblem("Fluxo_Maximo", LpMaximize)

# Função Objetiva
# Maximizar a capacidade de fluxo de saída do nó de origem
lista_fo = []
for key in var.keys():
  if key[0] == no_origem:
    lista_fo.append( var[key] )
model += lpSum(lista_fo)

# Restrições
for no in n_nos:
  if no != no_origem and no != no_destino :
    lista_entradas = []
    lista_saidas = []
    for aresta in var.keys():
      if aresta[0] == no:
        lista_saidas.append(var[aresta])
      elif aresta[1] == no:
        lista_entradas.append(var[aresta])

    # Todo nó que está no caminho ótimo deve ter uma entrada e uma saída
    model += lpSum(lista_entradas) == lpSum(lista_saidas)


In [12]:
print(model)
status = model.solve()
print(LpStatus[status])
print(f'O valor ótimo é {value(model.objective)}')
for i in var.keys():
  if(value(var[i]) != 0):
    print(f'{var[i]} = {value(var[i])}')

Fluxo_Maximo:
MAXIMIZE
1*x01 + 1*x02 + 1*x03 + 0
SUBJECT TO
_C1: x01 - x12 - x14 = 0

_C2: x02 + x12 - x23 - x24 + x32 = 0

_C3: x03 + x23 - x32 - x34 = 0

VARIABLES
x01 <= 200 Continuous
x02 <= 300 Continuous
x03 <= 100 Continuous
x12 <= 400 Continuous
x14 <= 300 Continuous
x23 <= 100 Continuous
x24 <= 200 Continuous
x32 <= 50 Continuous
x34 <= 200 Continuous

Optimal
O valor ótimo é 600.0
x01 = 200.0
x02 = 300.0
x03 = 100.0
x14 = 200.0
x23 = 100.0
x24 = 200.0
x34 = 200.0
